### These are some links that might help with writing the custom sampler:
- https://learning.oreilly.com/library/view/bayesian-data-analysis/9781439898222/OEBPS/pI.htm
- https://docs.pymc.io/notebooks/getting_started.html
- https://nbviewer.jupyter.org/gist/aflaxman/91d462301855d6942506
- https://docs.pymc.io/notebooks/sampling_compound_step.html

In [2]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import pymc3 as pm
import math as m

In [ ]:
from dipolarkernel import kernel

In [ ]:
## data loading
FileName = "3992_good_results.dat"

ImportedData = np.genfromtxt("./data/%s" % FileName, delimiter=',',skip_header=1)
t = ImportedData[:,0]
Vdata = ImportedData[:,1]

r = np.linspace(1,7,len(t))

# these values need to be loaded from a Tikhonov fit 
P_init = PfromTikhonov
delta_init = 1/alpha.^2


In [ ]:
# set up model
Vmax = np.amax(Vdata)*1.3
K = kernel(t,r)

L = regop(r,2) # this operator/functions needs to be written
LtL = np.conjugate(L)*L

with pm.Model() as gaussian_model: 

    ## Prior distributions as we had them before
    k = pm.Gamma('k',alpha=1,beta=0.05)

    sigma = pm.Gamma('sigma', alpha=1, beta=0.1)

    lambd = pm.Beta('lambda', alpha=1.3, beta=2.0)

    BoundedNormal = pm.Bound(pm.Normal,lower=0.0)
    V0 = BoundedNormal('V0', mu=1.0, sigma=0.2)

    # new prior for delta
    # this currently does not take into account P
    delta = pm.Gamma('delta',alpha = 0.01, beta = 1e-6)

    # some math operations
    tau = 1/sigma.^2

    tauKtK = tau*(np.conjugate(K)*K)

    S = K*P
    tauKtS = tau*np.conjugate(K)*S

    ## generate a P
    P_init = randP(delta_init,tauKtK,tauKtS,LtL,nt)

    ### This needs to be the manual sampler/step -----------------------------------
    invSigma = tauKtK + delta*LtL
    try
    #'lower' syntax is faster for sparse matrices. Also matches convention in Bardsley paper.
        C_L = chol(inv(invSigma),'lower')
    catch
        C_L = sqrtm(inv(invSigma))
    end
    v = randn(nt,1)
    w = C_L.'\v
    P = fnnls(invSigma,tauKtS+w)

    ##### ----------------------------------------------------------------------------


    ## Calculate signal
    Vmodel = V0*(1-lambd+lambd*S)*np.exp(-k*np.abs(t)))

    ## Likelihood
    V = pm.Normal('V', mu=Vmodel, sigma=sigma, observed=Vdata)

In [ ]:
## Sampling
with gaussian_model: 
    trace = pm.sample(draws=draws,tune=tune,chains=chains,cores=cores)


df_ = pm.trace_to_dataframe(trace)